# GAN(Generative Adversarial Networks) model

In [6]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split


In [45]:
data = np.genfromtxt("./oneD_power.csv", delimiter=",", skip_header=1)

# X = data[:, :-1]
X = data[:, -1]
y = data[:, :-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
# Scaler2 = StandardScaler()

# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.fit_transform(X_test)
X_scaled = X_train
y_scaled = y_train


In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(200000, 6)
(50000, 6)
(200000,)
(50000,)


# Model def

In [46]:
# hyperparameter setting
latent_dim = 100
output_dim = y.shape[1]

# Generator model
def build_generator():
    model = Sequential([
        Dense(128, input_dim=latent_dim+1),
        LeakyReLU(alpha=0.2),
        BatchNormalization(momentum=0.8),
        Dense(256),
        LeakyReLU(alpha=0.2),
        BatchNormalization(momentum=0.8),
        Dense(512),
        LeakyReLU(alpha=0.2),
        BatchNormalization(momentum=0.8),
        Dense(1, activation='linear')
    ])
    return model

# Discriminator model
def build_discriminator():
    model = Sequential([
        Dense(512, input_dim=1+1),
        LeakyReLU(alpha=0.2),
        Dense(256),
        LeakyReLU(alpha=0.2),
        Dense(1, activation='sigmoid')
    ])
    return model

# init
optimizer = Adam(0.0002, 0.5)
generator = build_generator()
discriminator = build_discriminator()

# Discriminator comfile
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Combined model (Generator + Discriminator)
z = Input(shape=(latent_dim,))
power_input = Input(shape=(1,))
gen_input = concatenate([z, power_input])
generated_param = generator(gen_input)

discriminator.trainable = False
validity = discriminator(concatenate([generated_param, power_input]))

combined = Model([z, power_input], validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

# Training

In [50]:
# Hyperparameter setting
epochs = 10 # 10000
batch_size = 64
sample_interval = 200

# training
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):
    idx = np.random.randint(0, X_scaled.shape[0], batch_size)
    power_batch = X_scaled[idx]
    real_params = y_scaled[idx]

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_params = generator.predict([noise, power_batch])

    d_loss_real = discriminator.train_on_batch(np.concatenate([real_params, power_batch], axis=1), valid)
    d_loss_fake = discriminator.train_on_batch(np.concatenate([gen_params, power_batch], axis=1), fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    g_loss = combined.train_on_batch([noise, power_batch], valid)

    if epoch % sample_interval == 0:
        print(f"{epoch} [D loss: {d_loss[0]}] [D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")

ValueError: in user code:

    File "/home/keem/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/home/keem/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/keem/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/home/keem/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/home/keem/anaconda3/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/keem/anaconda3/lib/python3.10/site-packages/keras/engine/input_spec.py", line 216, in assert_input_compatibility
    